<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/11-02-CNN-Ensenble-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.

# CNN -ensemble - MNIST



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

In [ ]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 128
drop_out = 0.3

In [ ]:
def get_model():
    inputs = keras.Input(shape=(28,28,1))

    # CL-PL #1
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = layers.MaxPool2D(pool_size=(2,2), strides=2, padding='same')(x)
    x = layers.Dropout(0.3)(x)

    # CL-PL #2
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(pool_size=(2,2), strides=2, padding='same')(x)
    x = layers.Dropout(0.3)(x)

    # CL-PL #3
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D(pool_size=(2,2), strides=2, padding='same')(x)
    x = layers.Dropout(0.3)(x)

    # FC #1
    flat = tf.reshape(x, [-1, 128 * 4 * 4])
    x = layers.Dense(units=625, activation='relu')(flat)
    x = layers.Dropout(0.5)(x)

    # FC #2
    outputs = layers.Dense(units=10, activation='softmax')(x)

    return keras.Model(inputs, outputs)

In [ ]:
model1 = get_model()
model2 = get_model()

In [ ]:
inputs = keras.Input(shape=(28,28,1))
y1 = model1(inputs)
y2 = model2(inputs)
outputs = layers.average([y1, y2])
ensemble_model = keras.Model(inputs=inputs, outputs=outputs)
ensemble_model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

In [ ]:
model1.summary()
model2.summary()
ensemble_model.summary()

In [ ]:
tf.keras.utils.plot_model(ensemble_model, show_shapes=True)

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print('x_train.shape:', x_train.shape, 'y_train.shape:', y_train.shape)

In [ ]:
x_test = x_test / 255
x_train = x_train / 255
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [ ]:
print('x_train.shape:', x_train.shape, 'x_test.shape:', x_test.shape)

In [ ]:
# one hot encode y data
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
print('y_train.shape:', y_train.shape)

In [ ]:
ensemble_model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs, verbose=0)

In [ ]:
# predict 10 random hand-writing data
y_predicted = ensemble_model.predict(x_test)
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    print("index: ", random_index,
          "actual y: ", np.argmax(y_test[random_index]),
          "predicted y: ", np.argmax(y_predicted[random_index]))

evaluation = ensemble_model.evaluate(x_test, y_test)
print('loss: ', evaluation[0])
print('accuracy', evaluation[1])